# Exemplo 07: Processamento e Leitura Paralela no Spark
## Contagem de Palavras Serial & Paralelo

A contagem de palavras em um texto é um dos exemplos mais comuns paa avaliar o desempenho de aplicações distribuídas. Ao contrário do exemplo de cálculo do número Pi que exige apenas processamento, a contagem de palavras exige a leitura de um arquivo texto. Assim, é possível avaliar o desempenho de aplicações paralelas com leitura de arquivo.

A contagem pode ser realizada através do paradgma MapReduce. A função Map em um pedaço do texto separa todas as palavras e contabiliza a ocorrencia de cada uma. A função Reduce irá juntar a contagem de todas as partes e totalizar o número de ocorrência de cada palavra.

Neste exemplo vamos contar as palavras do livro _Guerra e Paz_ de Tolstói (em ingles War and Peace, do projeto Gutemberg).

No primeiro caso, contamos em um único host, o segundo caso faz a contagem paralela em vários hosts usando virtualização JVM mas com uma partição única do arquivo, e o terceiro com o arquivo de entrada dividido em partes. O quarto caso faz a contagem  paralela em vários hosts usando virtualização Docker e o quinto também Docker com o arquivo de entrada dividido em partes.

Definir o número de partes no qual o arquivo será dividido é um desafio, mas os melhores resultados são obtidos quando o número de partes é maiso ou menos igual ao número de hosts. 

Lembramos que para executar um processamento distribuído com leitura de arquivo, este deverá estar disponível em todos os hosts. A melhor forma é colocaá-lo em um sistema de arquivos distribuído.

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
import time

In [3]:
# Configuration
# Processamento paralelo que le um arquivo necessita que ele esteja em todos os nós ou sistema de arquivos distribuidos
book_folder = "/data/dataset/books/english/"
book_file = "file://"+book_folder+"War_and_Peace.txt"
print(book_file)

file:///data/dataset/books/english/War_and_Peace.txt


## Word Count Serial

In [4]:
start_time = time.time()

# Create local Spark session
spark = SparkSession.builder \
        .appName("WordCountSerial") \
        .master("local[*]") \
        .getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [5]:
# Read de book file 
text_file = sc.textFile(book_file).cache()

counts = text_file.flatMap(lambda line: line.split()) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)

counts.toDF(["Word", "Count"]).sort("Count",ascending=False).show(5)

print("--- Execution time: %s seconds ---" % (time.time() - start_time))

+----+-----+
|Word|Count|
+----+-----+
| the|31704|
| and|20564|
|  to|16320|
|  of|14855|
|   a|10018|
+----+-----+
only showing top 5 rows

--- Execution time: 13.677977085113525 seconds ---


In [6]:
spark.stop()

## Word Count Parallel JVM

### Word Count JVM One Data Partition

In [7]:
# Starting Spark Cluster with JVM

spark = SparkSession.builder\
        .appName("WordCountParallel_JVM_1") \
        .master("mesos://zk://10.129.64.20:2181,10.129.64.10:2181,10.129.64.30:2181/mesos") \
        .getOrCreate()

spark.conf.set("spark.submit.deployMode", "client")
spark.conf.set("spark.mesos.containerizer","mesos")

# create the Spark Context
sc = spark.sparkContext

In [8]:
start_time = time.time()

# Read book file 
text_file = sc.textFile(book_file)

counts = text_file.flatMap(lambda line: line.split()) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)

counts.toDF(["Word", "Count"]).sort("Count",ascending=False).show(5)

print("--- Execution time: %s seconds ---" % (time.time() - start_time))

+----+-----+
|Word|Count|
+----+-----+
| the|31704|
| and|20564|
|  to|16320|
|  of|14855|
|   a|10018|
+----+-----+
only showing top 5 rows

--- Execution time: 7.442093849182129 seconds ---


In [9]:
spark.stop()

### Word Count JVM Five Data Partitions

In [10]:
# Starting Spark Cluster with JVM

spark = SparkSession.builder\
        .appName("WordCountParallel_JVM_5")\
        .master("mesos://zk://10.129.64.20:2181,10.129.64.10:2181,10.129.64.30:2181/mesos") \
        .getOrCreate()

spark.conf.set("spark.submit.deployMode", "client")
spark.conf.set("spark.mesos.containerizer","mesos")

# create the Spark Context
sc = spark.sparkContext

In [11]:
start_time = time.time()

# Read book file 
part = 5 
text_file_part = sc.textFile(book_file, part)

counts = text_file_part.flatMap(lambda line: line.split()) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)

counts.toDF(["Word", "Count"]).sort("Count",ascending=False).show(5)

print("--- Execution time: %s seconds ---" % (time.time() - start_time))

+----+-----+
|Word|Count|
+----+-----+
| the|31704|
| and|20564|
|  to|16320|
|  of|14855|
|   a|10018|
+----+-----+
only showing top 5 rows

--- Execution time: 7.0169336795806885 seconds ---


In [12]:
spark.stop()

## Word Count Parallel Docker

### Word Count Docker One Data Partition

In [13]:
# Starting Spark Cluster with Docker
spark = SparkSession.builder\
        .appName("JupyterPiParallel_Docker")\
        .master("mesos://zk://10.129.64.20:2181,10.129.64.10:2181,10.129.64.30:2181/mesos") \
        .config("spark.mesos.executor.docker.image","lasid/spark-worker") \
        .config("spark.mesos.containerizer","docker") \
        .getOrCreate()

spark.conf.set("spark.submit.deployMode", "client")

# create the Spark Context
sc = spark.sparkContext

In [14]:
start_time = time.time()

# Read book file 
text_file = sc.textFile(book_file)

counts = text_file.flatMap(lambda line: line.split()) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)

counts.toDF(["Word", "Count"]).sort("Count",ascending=False).show(5)

print("--- Execution time: %s seconds ---" % (time.time() - start_time))

+----+-----+
|Word|Count|
+----+-----+
| the|31704|
| and|20564|
|  to|16320|
|  of|14855|
|   a|10018|
+----+-----+
only showing top 5 rows

--- Execution time: 6.965694427490234 seconds ---


In [15]:
spark.stop()

### Word Count Docker Five Data Partitions

In [16]:
# Starting Spark Cluster with Docker
spark = SparkSession.builder\
        .appName("JupyterPiParallel_Docker")\
        .master("mesos://zk://10.129.64.20:2181,10.129.64.10:2181,10.129.64.30:2181/mesos") \
        .config("spark.mesos.executor.docker.image","lasid/spark-worker") \
        .config("spark.mesos.containerizer","docker") \
        .getOrCreate()

spark.conf.set("spark.submit.deployMode", "client")

# create the Spark Context
sc = spark.sparkContext

In [17]:
start_time = time.time()

# Read book file 
part = 5 
text_file_part = sc.textFile(book_file, part)

counts = text_file_part.flatMap(lambda line: line.split()) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)

counts.toDF(["Word", "Count"]).sort("Count",ascending=False).show(5)

print("--- Execution time: %s seconds ---" % (time.time() - start_time))

+----+-----+
|Word|Count|
+----+-----+
| the|31704|
| and|20564|
|  to|16320|
|  of|14855|
|   a|10018|
+----+-----+
only showing top 5 rows

--- Execution time: 7.08495306968689 seconds ---


In [18]:
spark.stop()